## Group 7
## Big Data Group Project

### Phase 2: Cleansing and Reshaping

This notebook is used for co-developing of codes required to wrangle the home credit risk dataset.

Please use this notebook as the live version and download local copies as you require.

If you have picked up a specific section, please comment your name to the code cell so that we can work on other sections.

Insert libraries into the cell below:

Insert code for data loading below:

Insert code for flagging missing values below:

Insert code for flagging duplicate values below:

Insert code for visualising or displaying outliers below:

Insert code for reviewing data values for inconsistencies below:

Insert code for reviewing data format below:

Insert code for encoding categorical variables below: